In [4]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/capstone-kitti-training/training/label_2/005063.txt
/kaggle/input/capstone-kitti-training/training/label_2/000637.txt
/kaggle/input/capstone-kitti-training/training/label_2/002604.txt
/kaggle/input/capstone-kitti-training/training/label_2/004229.txt
/kaggle/input/capstone-kitti-training/training/label_2/000360.txt
/kaggle/input/capstone-kitti-training/training/label_2/006439.txt
/kaggle/input/capstone-kitti-training/training/label_2/002979.txt
/kaggle/input/capstone-kitti-training/training/label_2/005938.txt
/kaggle/input/capstone-kitti-training/training/label_2/007127.txt
/kaggle/input/capstone-kitti-training/training/label_2/000870.txt
/kaggle/input/capstone-kitti-training/training/label_2/007419.txt
/kaggle/input/capstone-kitti-training/training/label_2/006003.txt
/kaggle/input/capstone-kitti-training/training/label_2/004343.txt
/kaggle/input/capstone-kitti-training/training/label_2/007478.txt
/kaggle/input/capstone-kitti-training/training/label_2/006338.txt
/kaggle/in

In [5]:
import os
from glob import glob
from pathlib import Path

from PIL import Image
from tqdm import tqdm

images_dir = '/kaggle/input/capstone-kitti-training/training/image_2' 
labels_dir = '/kaggle/input/capstone-kitti-training/training/label_2' 
output_dir = '/kaggle/working'

def generate_yolo_labels(
    images_dir: str,
    labels_dir: str,
    output_dir: str,
    classes: list = ["Car", "Pedestrian", "Cyclist"],
):
    """Generate YOLO labels."""
    print("Generating YOLO labels...")
    output_dir = os.path.join(output_dir, "labels")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    images = glob(os.path.join(images_dir, "*.png"))

    for image in tqdm(images, desc="Generating YOLO labels...", total=len(images)):
        frame = Image.open(image)
        image_name = Path(image).name
        image_id = image_name.split(".")[0]
        with open(os.path.join(labels_dir, f"{image_id}.txt"), "r") as f:
            labels = f.readlines()
        with open(os.path.join(output_dir, f"{image_id}.txt"), "w") as f:
            for label in labels:
                label = label.split(" ")
                _class = label[0]
                if _class not in classes:
                    continue
                class_id = classes.index(_class)
                xmin = float(label[4])
                ymin = float(label[5])
                xmax = float(label[6])
                ymax = float(label[7])
                x_center = (((xmax - xmin) / 2) + xmin) / frame.width
                y_center = (((ymax - ymin) / 2) + ymin) / frame.height
                w = (xmax - xmin) / frame.width
                h = (ymax - ymin) / frame.height
                if x_center > 1:
                    x_center = 1.0
                if y_center > 1:
                    y_center = 1.0
                if w > 1:
                    w = 1.0
                if h > 1:
                    h = 1.0
                f.write(f"{class_id} {x_center:.3f} {y_center:.3f} {w:.3f} {h:.3f}\n")


if __name__ == "__main__":
    generate_yolo_labels(
        images_dir,
        labels_dir,
        output_dir,
    )

Generating YOLO labels...


Generating YOLO labels...: 100%|██████████| 7481/7481 [03:10<00:00, 39.29it/s]


In [6]:
images_dir = '/kaggle/input/capstone-kitti-training/training/image_2'
labels_dir = '/kaggle/input/capstone-kitti-training/training/label_2' 
output_dir = '/kaggle/working'


train_val_split = 0.85
prefix = 'yolo'

images = glob(os.path.join(images_dir, "*.png"))
train_val_split = int(len(images) * train_val_split)
train_images = images[:train_val_split]
val_images = images[train_val_split:]
with open(os.path.join(output_dir, f"{prefix}_train.txt"), "w") as f:
    for image in train_images:
        f.write(f"./images/{Path(image).name}\n")
with open(os.path.join(output_dir, f"{prefix}_val.txt"), "w") as f:
    for image in val_images:
        f.write(f"./images/{Path(image).name}\n")


In [7]:
%pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [9]:
from typing import Union

from ultralytics import YOLO

config = "/kaggle/input/yolotraining/yolo_training.yaml"
epochs = 1
batch_size = 8
img_size = 640
device: Union[str, int] = 0
workers = 4
resume = False

# Load a model
model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(
    data=config,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    device=device,
    workers=workers,
)

model.export(format="onnx", half=fp16, dynamic=dynamic, simplify=True)



Ultralytics 8.3.55 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/kaggle/input/yolotraining/yolo_training.yaml, epochs=1, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=0, workers=4, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=T

RuntimeError: Dataset '/kaggle/input/yolotraining/yolo_training.yaml' error ❌ 
Dataset '/kaggle/input/yolotraining/yolo_training.yaml' images not found ⚠️, missing path '/kaggle/working/data/kitti/yolo_val.txt'
Note dataset download directory is '/kaggle/working/datasets'. You can update this in '/root/.config/Ultralytics/settings.json'